# MNIST with Uncertainty

In [1]:
import numpy as np
import tensorflow as tf

## Goal: Find $p(\mathbf{y}^*|\mathbf{x}^*, \mathbf{X}, \mathbf{Y})=\int{p(\mathbf{y}^*|\mathbf{x}^*, \mathbf{w}) p(\mathbf{w}|\mathbf{X}, \mathbf{Y})d\mathbf{w}}$
### Variational Inference
### $$q(\mathbf{y}^*|\mathbf{x}^*)=\int{p(\mathbf{y}^*|\mathbf{x}^*, \mathbf{w}) q(\mathbf{w})d\mathbf{w}}$$

## Uncertainties
## Let $y=f(\mathbf{x})+\epsilon$ where $\epsilon \sim \mathcal{N}(0, \sigma_a^2)$
### $$
\begin{align*}
\mathbb{E}[y-\hat{f}(\mathbf{x})]^2
&=\mathbb{E}[y-f(\mathbf{x})+f(\mathbf{x})-\hat{f}(\mathbf{x})]^2\\
&=\mathbb{E}[\epsilon+f(\mathbf{x})-\hat{f}(\mathbf{x})]^2\\
&=\mathbb{E}[\epsilon]^2+\mathbb{E}[f(\mathbf{x})-\hat{f}(\mathbf{x})]^2\\
&=\sigma_a^2 + \mathbb{E}[f(\mathbf{x})-\hat{f}(\mathbf{x})]^2\\
&=\sigma_a^2 + \sigma_e^2\\
\end{align*}
$$

### And, $$
\begin{align*}
\sigma_e^2 &= \mathbb{E}[f(\mathbf{x})-\hat{f}(\mathbf{x})]^2\\
&=\mathbb{E}[f(\mathbf{x})-\bar{f}(\mathbf{x})+\bar{f}(\mathbf{x})-\hat{f}(\mathbf{x})]^2\\
&=\mathbb{E}[f(\mathbf{x})-\bar{f}(\mathbf{x})]^2+\mathbb{E}[\bar{f}(\mathbf{x})-\hat{f}(\mathbf{x})]^2\\
&=bias(\hat{f}(\mathbf{x}))^2+Var(\hat{f}(\mathbf{x}))\\\\
&\text{Assuming $\hat{f}(\mathbf{x})$ is a unbiased estimator, we have}\\
&=Var(\hat{f}(\mathbf{x}))\\
&=\mathbb{E}[\hat{f}(\mathbf{x})^2]-\mathbb{E}[\hat{f}(\mathbf{x})]^2
\end{align*}
$$

### Hence, $$
\begin{align*}
\mathbb{E}[y-\hat{f}(\mathbf{x})]^2
&=\sigma_a^2 + \sigma_e^2\\
&=\sigma_a^2 + \mathbb{E}[\hat{f}(\mathbf{x})^2]-\mathbb{E}[\hat{f}(\mathbf{x})]^2
\end{align*}
$$

### We will estimate the uncertainties using MC Dropout as follows $$
\begin{align*}
\mathbb{E}[y-\hat{f}(\mathbf{x})]^2
&=\sigma_a^2 + \sigma_e^2\\
&=\sigma_a^2 + \mathbb{E}[\hat{f}(\mathbf{x})^2]-\mathbb{E}[\hat{f}(\mathbf{x})]^2\\
&\approx\hat{\sigma_a}^2 + \frac{1}{T}\sum_{t=1}^{T}{\hat{f_t}(\mathbf{x})^2}-\bigg(\frac{1}{T}\sum_{t=1}^{T}{\hat{f_t}(\mathbf{x})\bigg)}^2
\end{align*}
$$

### Sampling $$\mathbf{w}_t\sim q(\mathbf{w})$$

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

train_img = mnist.train._images
train_label = mnist.train._labels.copy()
test_img = mnist.test._images
test_label = mnist.test._labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Unseen label or Noise label

In [3]:
# 8 제거
# not_8 = np.where(train_label != 8)[0]
#
# train_img = train_img[not_8, :]
# train_label = train_label[not_8]

# 8에 노이즈
# idx_8 = np.where(train_label == 8)[0]
# train_label[idx_8] = np.random.choice(np.arange(10), idx_8.shape[0])

### Hyperparameter

In [4]:
bn_moment = 0.9
n_mc_dropout = 50 # Dropout의 횟수

batch_size_tr = 32
batch_size_ts = 64
model_name = 'MNIST_Uncertainty'

### Weight initializer & Regularizer

In [5]:
weight_initializer = tf.contrib.layers.variance_scaling_initializer()
regularizer = tf.contrib.layers.l2_regularizer(10**-4)

In [6]:
x = tf.placeholder(tf.float32, [None, 784], name='x')
y = tf.placeholder(tf.int32, [None], name='target')

training = tf.placeholder(tf.bool, name='training')
conv_keep_prob = tf.placeholder(tf.float32, name='conv_keep_prob')
fc_keep_prob = tf.placeholder(tf.float32, name='fc_keep_prob')

### Copy input data for MC integration

In [7]:
repeat = tf.transpose(tf.tile(tf.expand_dims(x, 0), [n_mc_dropout, 1, 1]), [1, 0, 2])
x_img = tf.reshape(repeat, [-1, 28, 28, 1])

y_repeat = tf.transpose(tf.tile(tf.expand_dims(y, 0), [n_mc_dropout, 1]))
y_repeat = tf.reshape(y_repeat, [-1])

## Network architecture

### Convolution Layers

In [8]:
with tf.variable_scope('conv1'):
    net = tf.layers.conv2d(x_img, 32, kernel_size=[3, 3], strides=[2, 2], padding='SAME',
                           kernel_initializer=weight_initializer,
                           kernel_regularizer=regularizer, bias_regularizer=regularizer)
    net = tf.layers.batch_normalization(net, momentum=bn_moment, training=training)
    net = tf.nn.relu(net)
    net = tf.layers.dropout(net, conv_keep_prob, training=training)

with tf.variable_scope('conv2'):
    net = tf.layers.conv2d(net, 64, kernel_size=[3, 3], padding='SAME',
                           kernel_initializer=weight_initializer,
                           kernel_regularizer=regularizer, bias_regularizer=regularizer)
    net = tf.layers.batch_normalization(net, momentum=bn_moment, training=training)
    net = tf.nn.relu(net)
    net = tf.layers.dropout(net, conv_keep_prob, training=training)

with tf.variable_scope('conv3'):
    net = tf.layers.conv2d(net, 64, kernel_size=[3, 3], padding='SAME',
                           kernel_initializer=weight_initializer,
                           kernel_regularizer=regularizer, bias_regularizer=regularizer)
    net = tf.layers.batch_normalization(net, momentum=bn_moment, training=training)
    net = tf.nn.relu(net)
    net = tf.layers.dropout(net, conv_keep_prob, training=training)

with tf.variable_scope('conv4'):
    net = tf.layers.conv2d(net, 128, kernel_size=[3, 3], strides=[2, 2], padding='SAME',
                           kernel_initializer=weight_initializer,
                           kernel_regularizer=regularizer, bias_regularizer=regularizer)
    net = tf.layers.batch_normalization(net, momentum=bn_moment, training=training)
    net = tf.nn.relu(net)
    net = tf.layers.dropout(net, conv_keep_prob, training=training)

### Fully Connected Layers

In [9]:
with tf.variable_scope('fc1'):
    net = tf.layers.conv2d(net, 512, kernel_size=[7, 7], padding='VALID',
                           kernel_initializer=weight_initializer,
                           kernel_regularizer=regularizer, bias_regularizer=regularizer)
    net = tf.layers.batch_normalization(net, momentum=bn_moment, training=training)
    net = tf.nn.relu(net)
    net = tf.layers.dropout(net, fc_keep_prob, training=training)

with tf.variable_scope('fc2'):
    net = tf.layers.conv2d(net, 512, kernel_size=[1, 1], padding='VALID',
                           kernel_initializer=weight_initializer,
                           kernel_regularizer=regularizer, bias_regularizer=regularizer)
    net = tf.layers.batch_normalization(net, momentum=bn_moment, training=training)
    net = tf.nn.relu(net)
    net = tf.layers.dropout(net, fc_keep_prob, training=training)

### Noise sampling

In [10]:
with tf.variable_scope('logit'):
    net = tf.layers.conv2d(net, 11, kernel_size=[1, 1], padding='VALID',
                           kernel_initializer=weight_initializer,
                           kernel_regularizer=regularizer, bias_regularizer=regularizer)
    net = net[:, 0, 0, :]
    
    mu = net[..., :10]
    s = net[..., -1]
    sigma = tf.exp(s/2)
    target_shape = tf.shape(mu)
    
    mc_out = mu + tf.random_normal(target_shape)*tf.expand_dims(sigma, -1)

### Compute Loss

According to [the paper](https://arxiv.org/abs/1703.04977), loss has to be calculated as follow.
\begin{align*}
p(y=c|\mathbf{x}, \mathbf{X}, \mathbf{Y})
&=\int{Softmax(f_\mathbf{w}(\mathbf{x}))p(\mathbf{w}|\mathbf{X}, \mathbf{Y})d\mathbf{w}}\\
&\approx \int{Softmax(f_\mathbf{w}(\mathbf{x}))q(\mathbf{w})d\mathbf{w}}\\
&\approx \frac{1}{T}\sum_{t=1}^{T}{Softmax(f_{\mathbf{w}_t}(\mathbf{x}))}
\end{align*}
Sampling $$\mathbf{w}_t \sim q(\mathbf{w})$$

Hence, $$
\begin{align*}
\mathcal{L}=-\sum_i{\log{\frac{1}{T}\sum_t{Softmax\big(f_{\mathbf{w}_t}(\mathbf{x})\big)}}}
\end{align*}
$$

But, following code minimize
$$
\mathcal{L^{alt}}=-\sum_i{\frac{1}{T}\sum_t{\log{Softmax\big(f_{\mathbf{w}_t}(\mathbf{x})\big)}}}
$$
with weight decay.

In [11]:
with tf.variable_scope('loss'):
    # prob = tf.reduce_mean((tf.nn.softmax(mc_out), 0)

    # idx = tf.concat([tf.expand_dims(tf.range(target_shape[0]*target_shape[1]), -1), tf.expand_dims(y_repeat, -1)], -1)
    # loss = tf.gather_nd(tf.reduce_logsumexp(mc_out - tf.expand_dims(tf.reduce_logsumexp(mc_out, -1), -1), 1), idx)
    # loss -= tf.log(float(n_mc_dropout)) # 학습상 상수이기에 필요 없다.

    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_repeat, logits=mc_out)

    loss = tf.reduce_mean(loss)
    tf.summary.scalar('cross_entropy', loss)

    loss += tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    tf.summary.scalar('total_loss', loss)


with tf.name_scope('optimizer'):
    global_step = tf.Variable(0, trainable=False, name='global_step')
    learning_rate = tf.placeholder(tf.float32)
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        rmsprop = tf.train.RMSPropOptimizer(learning_rate).minimize(loss, global_step=global_step)
        sgd = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

### Compute Uncertainties
\begin{align*}
\mathbb{E}[y-\hat{f}(\mathbf{x})]^2
&=\sigma_a^2 + \sigma_e^2\\
&=\sigma_a^2 + \mathbb{E}[\hat{f}(\mathbf{x})^2]-\mathbb{E}[\hat{f}(\mathbf{x})]^2\\
&\approx\hat{\sigma_a}^2 + \frac{1}{T}\sum_{t=1}^{T}{\hat{f_t}(\mathbf{x})^2}-\bigg(\frac{1}{T}\sum_{t=1}^{T}{\hat{f_t}(\mathbf{x})\bigg)}^2
\end{align*}


In [12]:
with tf.name_scope('uncertainties'):
    mu = tf.reshape(mu, [-1, 50, 10])
    sigma = tf.reshape(sigma, [-1, 50])
    epistemic = tf.reduce_mean(mu**2, 1) - tf.reduce_mean(mu, 1)**2
    aleatoric = tf.reduce_mean(sigma**2, 1)

\begin{align*}
p(y=c|\mathbf{x}, \mathbf{X}, \mathbf{Y})
&=\int{Softmax(f_\mathbf{w}(\mathbf{x}))p(\mathbf{w}|\mathbf{X}, \mathbf{Y})d\mathbf{w}}\\
&\approx \int{Softmax(f_\mathbf{w}(\mathbf{x}))q(\mathbf{w})d\mathbf{w}}\\
&\approx \frac{1}{T}\sum_{t=1}^{T}{Softmax(f_{\mathbf{w}_t}(\mathbf{x}))}
\end{align*}
Sampling $$\mathbf{w}_t \sim q(\mathbf{w})$$

In [13]:
with tf.name_scope('y_prob'):
    # expected_prob = tf.reduce_mean(tf.exp(mc_out - tf.expand_dims(tf.reduce_logsumexp(mc_out, -1), -1)), 1)

    mc_out_reshape = tf.reshape(mc_out, [-1, 50, 10])
    expected_prob = tf.reduce_mean(tf.exp(mc_out_reshape - tf.expand_dims(tf.reduce_logsumexp(mc_out_reshape, -1), -1)), 1)

    y_pred = tf.argmax(expected_prob, 1, output_type=tf.int32)
    tf.summary.histogram('probability', expected_prob)
    correct_prediction = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

### Training

In [14]:
merged = tf.summary.merge_all()

def feed_dict(train, batch_size, lr=0.01):
    if train:
        batch_idx = np.random.choice(train_img.shape[0], batch_size, False)
        xs = train_img[batch_idx, :]
        ys = train_label[batch_idx]
        tr = True
    else:
        batch_idx = np.random.choice(test_img.shape[0], batch_size, False)
        xs = test_img[batch_idx, :]
        ys = test_label[batch_idx]
        tr = True
    return {x: xs, y: ys, training: tr, learning_rate: lr, conv_keep_prob: 0.8, fc_keep_prob: 0.8}



config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())


train_writer = tf.summary.FileWriter(model_name + '/train',sess.graph)
test_writer = tf.summary.FileWriter(model_name + '/test')

optimizer = rmsprop

In [15]:
i = 1

lr = 0.01
while (i <= (3000)):
    if i % 10 == 0:
        summary, tr = sess.run([merged, optimizer], feed_dict=feed_dict(True, batch_size_tr, lr))
        train_writer.add_summary(summary, i)
        summary, acc, bat_loss = sess.run([merged, accuracy, loss], feed_dict=feed_dict(False, batch_size_ts))
        test_writer.add_summary(summary, i)
        print('Accuracy and loss at step %s: %s %s' % (i, acc, bat_loss))
    else:
        tr = sess.run(optimizer, feed_dict=feed_dict(True, batch_size_tr, lr))
    i += 1

Accuracy and loss at step 10: 0.03125 4.27133
Accuracy and loss at step 20: 0.125 3.97181
Accuracy and loss at step 30: 0.09375 3.73276
Accuracy and loss at step 40: 0.109375 3.54113
Accuracy and loss at step 50: 0.0625 3.50008
Accuracy and loss at step 60: 0.125 3.01057
Accuracy and loss at step 70: 0.09375 3.24462
Accuracy and loss at step 80: 0.09375 3.20744
Accuracy and loss at step 90: 0.21875 2.81554
Accuracy and loss at step 100: 0.203125 2.80749
Accuracy and loss at step 110: 0.140625 2.61759
Accuracy and loss at step 120: 0.4375 2.03378
Accuracy and loss at step 130: 0.625 1.53541
Accuracy and loss at step 140: 0.71875 1.49552
Accuracy and loss at step 150: 0.5625 1.5717
Accuracy and loss at step 160: 0.65625 1.47854
Accuracy and loss at step 170: 0.765625 1.26963
Accuracy and loss at step 180: 0.71875 1.58199
Accuracy and loss at step 190: 0.890625 1.02184
Accuracy and loss at step 200: 0.828125 1.15517
Accuracy and loss at step 210: 0.765625 1.57003
Accuracy and loss at step

Accuracy and loss at step 1740: 0.921875 1.06051
Accuracy and loss at step 1750: 0.953125 0.882551
Accuracy and loss at step 1760: 0.984375 0.906187
Accuracy and loss at step 1770: 0.890625 1.16588
Accuracy and loss at step 1780: 0.921875 1.01362
Accuracy and loss at step 1790: 0.953125 1.05884
Accuracy and loss at step 1800: 0.9375 1.07889
Accuracy and loss at step 1810: 0.96875 0.867563
Accuracy and loss at step 1820: 0.96875 0.918563
Accuracy and loss at step 1830: 0.921875 0.971684
Accuracy and loss at step 1840: 0.96875 0.888339
Accuracy and loss at step 1850: 0.921875 1.07853
Accuracy and loss at step 1860: 0.921875 1.08424
Accuracy and loss at step 1870: 0.953125 0.878093
Accuracy and loss at step 1880: 0.953125 0.920796
Accuracy and loss at step 1890: 0.953125 0.97094
Accuracy and loss at step 1900: 0.96875 0.972466
Accuracy and loss at step 1910: 0.953125 0.901757
Accuracy and loss at step 1920: 0.953125 1.00808
Accuracy and loss at step 1930: 0.953125 0.888874
Accuracy and lo